In [5]:
import requests
import os
import json
import time
import pandas as pd
import numpy as np

from datetime import datetime
from pytz import timezone


#Academic申請後に生成される"Bearer Token(BT)"
BT = '###############'


TWEET_LIMIT_you_set = 1000 #### Defult = 1000 for test


search_url = "https://api.twitter.com/2/tweets/search/all"

#与えられたパラメータからツイート取得条件(query_params)を生成する関数
def make_parm(keyword, start_time, end_time):

    #検索条件(今回はキーワード、開始時刻、修了時刻のみ)
    query_params = {'query': keyword ,
                    'tweet.fields': 'created_at,referenced_tweets',
                    'expansions': 'author_id,geo.place_id,in_reply_to_user_id',
                    'start_time': start_time,
                    'end_time': end_time,
                    'user.fields': 'created_at,description,location,username',
                    'max_results':500,  #一回のqueryは５００で上限らしい
                    'next_token' : {} #次のページにいくためのparam
                   }
    return query_params


def create_headers(BT):
    headers = {"Authorization": "Bearer {}".format(BT)}
    return headers


def connect_to_endpoint(url, headers, params):
    response = requests.request("GET", search_url, headers=headers, params=params)
    #print(response.status_code)
    if response.status_code != 200:
        #raise Exception(response.status_code, response.text)
    
        print(response.status_code, response.text)
        
    return response   ######## to make use of  header information, return response instead of response.json()

def convert_timezone(normalized_data):
    normalized_data = normalized_data.copy().dropna(subset=['created_at'])
    normalized_data.created_at = pd.to_datetime(normalized_data.created_at, utc=True)
    normalized_data.index = pd.DatetimeIndex(normalized_data.created_at, name='created_at')
    normalized_data.index = normalized_data.index.tz_convert('Asia/Tokyo')
    
    # sort dataframe by created time
    
    normalized_data.created_at = normalized_data.index
    normalized_data = normalized_data.sort_index()
    normalized_data = normalized_data.reset_index(drop=True)
    
    
    # rename index
    
    print('Timezone convertion finished')
    return normalized_data

def store_checkpoint(normalized_data, keyword):
    
    # store collected data to csv in case of error occurrence
    
    savedir = './' # set your savedir here
    
    kwindex = keyword

        
    filename = kwindex + normalized_data.iloc[0]['created_at'].strftime('%Y%m%d%H%M') + '-' + \
               normalized_data.iloc[-1]['created_at'].strftime('%Y%m%d%H%M') + \
                '.csv'


    savepath = savedir + filename    
    
    normalized_data.to_csv(path_or_buf = savepath)
    print('Successfully saved at ' + savepath + ' at ' + \
              datetime.today().strftime("%Y/%m/%d-%H:%M"))
    
def json_to_df(json_response):
    
    #print(json_response)
    normalized_data = pd.json_normalize(json_response['data'])
    normalized_users = pd.json_normalize(json_response['includes']['users']).set_index('id')
     
    normalized_users.rename({'id': 'author_id'}, inplace = True, axis = 1)
    normalized_data.rename({'geo.place_id': 'place_id'}, inplace = True, axis = 1)
            
    normalized = normalized_data.join(normalized_users, on = 'author_id', how = 'outer', rsuffix = '_user')
    normalized['text'] = normalized['text'].str.replace('\n', ' ')     #text内の\nを削除
    normalized['text'] = normalized['text'].str.replace('@', ' @')     #@前にスペースを追加
    
    normalized['referenced_tweets_type'] = normalized['referenced_tweets'].apply(lambda x: x[0]['type'] if (np.all(pd.notnull(x))) else np.nan)
    normalized['referenced_tweets_id'] = normalized['referenced_tweets'].apply(lambda x: x[0]['id'] if (np.all(pd.notnull(x))) else np.nan)
    del normalized['referenced_tweets']

            
    return normalized


#実際にツイート取得をする関数(上の5つの関数を呼び出しながら)
def main(keyword, start_time, end_time):

    count = 0
    flag = True
    TWEET_LIMIT = TWEET_LIMIT_you_set
    
    
    normalized_data_old = pd.DataFrame(columns=['created_at','id', 'text', 'author_id', 
                                            'referenced_tweets_type',  'referenced_tweets_id',
                                            'in_reply_to_user_id' ,
                                            'place_id', 'description',
                                            'name', 'verified', 'username', 'created_at_user',
                                            'location'])
    
    #cols = ['created_at', 'id', ' text']
    #normalized_data_old = pd.DataFrame(index=[], columns=cols)#create checkpoint dataframe
    
    query_params = make_parm(keyword, start_time, end_time)#ここで取得条件が返される
    
    while flag:
        if count >= TWEET_LIMIT:
            break
    
        headers = create_headers(BT)
        time.sleep(1)
        
        response = connect_to_endpoint(search_url, headers, query_params)
        
        json_response = response.json()  
        
        
        #print(json_response['data']) 
        #print(json_response) 
        
        normalized_data = json_to_df(json_response)
        
        normalized_data_new = pd.concat([normalized_data_old, normalized_data], ignore_index=True)
         
        normalized_data_old = normalized_data_new
        
        print("total:" + str(len(normalized_data_old)) + "tweets")
    

        result_count = json_response['meta']['result_count']
        
        if 'next_token' in json_response['meta']:
            next_token = json_response['meta']['next_token']
            query_params['next_token'] = next_token
            count += result_count
            
            time.sleep(3)
            
            response = connect_to_endpoint(search_url, headers, query_params)
            time.sleep(3)
            
            
            if response.status_code == 200:
                
                continue
            
            elif response.status_code == 429:
                
                # store checkpoints when error occurs
                
                normalized_data_last = convert_timezone(normalized_data_old)
                store_checkpoint(normalized_data_last, keyword)
                
                # from https://github.com/mammalofski/Twitter-Scraper/blob/main/Scraper.py
                
                print('too many requests ... ')
                print('the header is ', response.headers)
                throttle_end_timestamp = int(response.headers.get('x-rate-limit-reset'))
                throttle_end_time = datetime.strftime(datetime.fromtimestamp(throttle_end_timestamp), "%H:%M:%S")
                time_to_wait = int(throttle_end_timestamp - datetime.now().timestamp()) + 5
                print('lets rest for', time_to_wait, 'seconds and wake up at', throttle_end_time)
                print('sleeping ...')
                time.sleep(time_to_wait)
                response = connect_to_endpoint(search_url, headers, query_params)
                
            else:
                normalized_data_last = convert_timezone(normalized_data_old)
                store_checkpoint(normalized_data_last, keyword)
                
                print('un expected error : ')
                print(response.status_code, response.text)
                print('sleep for 15 min')
                time.sleep( 15 * 60 + 10)  # rest 15min 
                print('restart at ' + datetime.strftime(datetime.now(), "%Y-%m-%d %H:%M:%S"))
                
                response = connect_to_endpoint(search_url, headers, query_params)
             
            time.sleep(3)  # rate limit = 1 request/1 sec
            
            json_response = response.json()
            
            
            #append the current page of results to lists
            
            
            normalized_data = json_to_df(json_response)
        
            normalized_data_new = pd.concat([normalized_data_old, normalized_data], ignore_index=True)
            
            normalized_data_old = normalized_data_new
            
            print("total:" + str(len(normalized_data_old)) + "tweets")

        else:
            flag = False
            print('Last page of query results')
            
            
    
    # convert UTC +0 timezone to UTC +9 timezone
    normalized_data_last = convert_timezone(normalized_data_old)
    
    if count % 500 <= 100:
        print('Finished collecting tweets at ' + str(normalized_data_last.iloc[-1]['created_at']))
    
    
    # store checkpoint to csv
    
    store_checkpoint(normalized_data_last, keyword)
    
    #normalized_data_last.to_csv('../FullArchiveData_2021Apr/T20200331-0501.csv')#csvファイルに保存

In [2]:
main(keyword="研究楽しい", start_time="2022-03-15T00:00:00Z", end_time="2022-03-16T00:00:00Z")

total:12tweets
Last page of query results
Timezone convertion finished
Finished collecting tweets at 2022-03-16 03:51:57+09:00
Successfully saved at ./研究楽しい202203151321-202203160351.csv at 2022/03/22-20:26


In [3]:
test = pd.read_csv('./研究楽しい202203151321-202203160351.csv',usecols=['created_at','id', 'text', 'author_id'],dtype={'id':str})

In [4]:
test

,created_at,id,text,author_id
0,2022-03-15 13:21:26+09:00,1503587147898830852,自分の顔に飽きてきたけど、大きく整形したい所は無いから、ウィッグや普段着ない服買ってみた。 ...,1279852314195443712
1,2022-03-15 16:33:14+09:00,1503635417068826628,みんなからの匿名質問を募集中！ こんな質問に答えてるよ ● sです ● ずばりイメージのい...,845446421109125121
2,2022-03-15 17:25:23+09:00,1503648543449714689,@maru_i5 友達にシャドウで引いてみたら？ってアドバイスされてやってみたの🥰🥰 メイ...,845613959432421376
3,2022-03-15 20:33:19+09:00,1503695836622254083,@Fkga_34 メイク研究楽しい変われるから楽しい˙𐃷˙,1383084669881618433
4,2022-03-15 22:11:52+09:00,1503720639818973184,ソーヴァ研究楽しい,2441593783
5,2022-03-15 22:13:23+09:00,1503721020758573058,研究→楽しい。,2479324278
6,2022-03-15 22:34:50+09:00,1503726416088924162,@mabo05201 山ご飯研究楽しいですよね(๑˃̵ᴗ˂̵),1428702171646685185
7,2022-03-15 23:31:06+09:00,1503740575795003394,ソーヴァのダーツ研究楽しいけど実践で使えるかは不安。,852784310738341889
8,2022-03-16 03:51:57+09:00,1503806224206217216,マップ研究楽しいけど、車湧きとかルート取りとかまで考えるとマジで脳疲れる。 改めてベネキさ...,1333503066373427200
